In [5]:
import pandas as pd
import re

In [2]:
# load up transposed tunes
df = pd.read_csv('data/transposed_tunes.csv')

In [3]:
df.head()

,Unnamed: 0.1,Unnamed: 0,tune_id,setting_id,name,type,meter,mode,abc,date,username,full_abc,transposed_abc
0,0,0,18105,35234,"$150 Boot, The",polka,2/4,Gmajor,|:d>g fe|dB AG|E/F/G E/F/G|BA GF|\nd>g fe|dB A...,2019-07-06 04:39:09,NfldWhistler,"X:35234\nT:$150 Boot, The\nM:2/4\nK:Gmajor\n|:...","X:35234\nT:$150 Boot, The\nM:2/4\nK:Cmajor\n|:..."
1,1,1,11931,11931,'G Iomain Nan Gamhna,slip jig,9/8,Gmajor,dBB B2 A BAG|dBB Bcd efg|dBB B2 A BAG|eAA dBG ...,2012-05-17 07:49:26,iliketurtles,X:11931\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Gmaj...,X:11931\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Cmaj...
2,2,2,11931,48772,'G Iomain Nan Gamhna,slip jig,9/8,Amixolydian,|:dBB BBA BAG|dBB Bcd efg|dBB BBA BAG|e2A dBG ...,2023-11-25 22:54:00,birlibirdie,X:48772\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Amix...,X:48772\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Cmix...
3,3,3,15326,28560,'S Ann An Ìle,strathspey,4/4,Gmajor,|:G>A B>G c>A B>G|E<E A>G F<D D2|G>A B>G c>A B...,2016-03-31 15:34:45,danninagh,X:28560\nT:'S Ann An Ìle\nM:4/4\nK:Gmajor\n|:G...,X:28560\nT:'S Ann An Ìle\nM:4/4\nK:Cmajor\n|:C...
4,4,4,15326,28582,'S Ann An Ìle,strathspey,4/4,Gmajor,"uD2|:{F}v[G,2G2]uB>ud c>A B>G|{D}E2 uA>uG F<D ...",2016-04-03 09:15:08,DonaldK,X:28582\nT:'S Ann An Ìle\nM:4/4\nK:Gmajor\nuD2...,"X:28582\nT:'S Ann An Ìle\nM:4/4\nK:Cmajor\nuG,..."


In [6]:
df.transposed_abc[34]

'X:9294\nT:18-6 Two Step\nM:6/8\nK:Cmajor\nA^GA dcA|G^FG c3|DEF E2 D|C3 z2G|\n|:G^FG AGF|GAG E3|G^FG AGE|F3 z3|\nG,^G,A, B,=B,C|D2 E F2 ^F|G^FG AGF|E3 z3|\nG^FG AGF|GAG E3|G^FG c2 B|A3 z3|\nA^GA dcA|G^FG c3|DEF E2 D|1 C3 z2 G:|2 C3 z2 D||\n|:G2 D A2 D|B3 G2 ^F|E^DE G2 E|D3 z2 D|\nG2 D A2 D|B3 G2 B|A2 E ^F2 G|A3 z2 D|\nG2 D A2 D|B3 G2 ^F|E2 ^G A2 B|c3 z3|\nE^DE G^FE|DGB d2 c|BAG D2 ^F|1 G3 z2 D:|2 G3 z2 G||\nG^FG AGF|GAG E3|G^FG AGE|F3 z3|\nG,^G,A, B,=B,C|D2 E F2 ^F|G^FG AGF|E3 z3|\nG^FG AGF|GAG E3|GAG c2 B|A3 z3|\nA^GA dcA|G^FG c3|DEF E2 D|C3 z2 B,||\nA,3 ^G,2 A,|F,6|A,^G,A, DCA,|B,6|\nG,3 ^F,2 G,|E,6|C,D,E, F,G,^G,|A,6|\nA,3 ^G,2 A,|F,6|FEF G2 F|D6|\nD^CD FED|C=B,C F3|G,A,B, A,2 G,|F,3 G3||'

In [8]:
def infer_unit_length(header: dict) -> float:
    if 'L' in header:
        return eval(header['L'])  # e.g., "1/8" → 0.125
    m = header.get('M', '4/4')
    if m == '6/8':
        return 1/8
    return 1/8  # reasonable default
note_pattern = re.compile(r"([=^_]*[A-Ga-gz])(\d*/*\d*)?")
def abc_length_to_float(length_str):
    if not length_str:
        return 1.0
    if '/' in length_str:
        if length_str == '/':
            return 0.5
        parts = length_str.split('/')
        if len(parts) == 2 and parts[0] and parts[1]:
            return float(parts[0]) / float(parts[1])
        elif parts[0]:
            return float(parts[0]) / 2
        else:
            return 1.0 / float(parts[1])
    else:
        return float(length_str)


In [12]:
HEADER_RE = re.compile(r'^(X|T|M|L|K):\s*(.+)$', re.MULTILINE)
NOTE_RE = re.compile(r'([=^_]*[A-Ga-gzZx])([0-9/]*)')

# Heuristic unit length mapping based on meter denominator (e.g., 4 in 4/4)
METER_BASE_UNIT = {
    2: 1/8,
    4: 1/8,
    8: 1/8,
    16: 1/16,
    3: 1/8,   # for 3/4, 3/8 jigs, etc.
    9: 1/8,   # 9/8 slip jigs
    12: 1/8,  # 12/8 slides, etc.
    6: 1/8,
    5: 1/8   # fallback
}


def extract_header(abc_str):
    header = {}
    for match in HEADER_RE.finditer(abc_str):
        field, value = match.groups()
        header[field] = value.strip()
    return header


def infer_unit_length(header):
    if 'L' in header:
        try:
            return eval(header['L'])
        except Exception:
            pass
    meter = header.get('M', '4/4')
    try:
        denominator = int(meter.split('/')[-1])
        return METER_BASE_UNIT.get(denominator, 1/8)
    except Exception:
        return 1/8  # fallback


def abc_length_to_float(length_str):
    if not length_str:
        return 1.0
    if '/' in length_str:
        if length_str == '/':
            return 0.5
        parts = length_str.split('/')
        try:
            if len(parts) == 2 and parts[0] and parts[1]:
                return float(parts[0]) / float(parts[1])
            elif parts[0]:
                return float(parts[0]) / 2
            elif parts[1]:
                return 1.0 / float(parts[1])
            else:
                return 1.0  # fallback for malformed /
        except ValueError:
            return 1.0  # fallback if conversion fails
    else:
        try:
            return float(length_str)
        except ValueError:
            return 1.0  # fallback


def tokenize_abc_notes(abc_str):
    """
    Tokenizes ABC into [note_duration] format, where duration is in quarter notes.
    """
    header = extract_header(abc_str)
    unit_length = infer_unit_length(header)  # e.g. 1/8 = 0.125
    body = abc_str.split('\n')

    # Skip header lines
    body = [line for line in body if not HEADER_RE.match(line)]
    body_str = ' '.join(body)

    tokens = []
    for match in NOTE_RE.finditer(body_str):
        pitch, length = match.groups()
        duration = abc_length_to_float(length) * (unit_length / 0.25)
        pitch = pitch.replace('Z', 'z').replace('x', 'z')  # normalize rests
        token = f"{pitch}_{duration:.3f}"
        tokens.append(token)

    return tokens


In [14]:
# Keep only rows where transposed_abc is a string
df = df[df['transposed_abc'].apply(lambda x: isinstance(x, str))]
df['tokenized_abc'] = df['transposed_abc'].apply(tokenize_abc_notes)

In [15]:
df.head()

,Unnamed: 0.1,Unnamed: 0,tune_id,setting_id,name,type,meter,mode,abc,date,username,full_abc,transposed_abc,tokenized_abc
0,0,0,18105,35234,"$150 Boot, The",polka,2/4,Gmajor,|:d>g fe|dB AG|E/F/G E/F/G|BA GF|\nd>g fe|dB A...,2019-07-06 04:39:09,NfldWhistler,"X:35234\nT:$150 Boot, The\nM:2/4\nK:Gmajor\n|:...","X:35234\nT:$150 Boot, The\nM:2/4\nK:Cmajor\n|:...","[G_0.500, c_0.500, B_0.500, A_0.500, G_0.500, ..."
1,1,1,11931,11931,'G Iomain Nan Gamhna,slip jig,9/8,Gmajor,dBB B2 A BAG|dBB Bcd efg|dBB B2 A BAG|eAA dBG ...,2012-05-17 07:49:26,iliketurtles,X:11931\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Gmaj...,X:11931\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Cmaj...,"[G_0.500, E_0.500, E_0.500, E_1.000, D_0.500, ..."
2,2,2,11931,48772,'G Iomain Nan Gamhna,slip jig,9/8,Amixolydian,|:dBB BBA BAG|dBB Bcd efg|dBB BBA BAG|e2A dBG ...,2023-11-25 22:54:00,birlibirdie,X:48772\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Amix...,X:48772\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Cmix...,"[F_0.500, D_0.500, D_0.500, D_0.500, D_0.500, ..."
3,3,3,15326,28560,'S Ann An Ìle,strathspey,4/4,Gmajor,|:G>A B>G c>A B>G|E<E A>G F<D D2|G>A B>G c>A B...,2016-03-31 15:34:45,danninagh,X:28560\nT:'S Ann An Ìle\nM:4/4\nK:Gmajor\n|:G...,X:28560\nT:'S Ann An Ìle\nM:4/4\nK:Cmajor\n|:C...,"[C_0.500, D_0.500, E_0.500, C_0.500, F_0.500, ..."
4,4,4,15326,28582,'S Ann An Ìle,strathspey,4/4,Gmajor,"uD2|:{F}v[G,2G2]uB>ud c>A B>G|{D}E2 uA>uG F<D ...",2016-04-03 09:15:08,DonaldK,X:28582\nT:'S Ann An Ìle\nM:4/4\nK:Gmajor\nuD2...,"X:28582\nT:'S Ann An Ìle\nM:4/4\nK:Cmajor\nuG,...","[G_0.500, B_0.500, C_0.500, C_1.000, E_0.500, ..."


In [16]:
df.to_csv('data/tokenized_abc.csv')

In [18]:
# time to generate n-gramsdef generate_ngrams(tokens, n=3):
def generate_ngrams(tokens, n=3):
    """
    Converts a list of tokens into sliding n-grams of size `n`.
    Example: ['A_0.5', 'B_0.5', 'C_0.5'] → ['A_0.5 B_0.5 C_0.5']
    """
    return [' '.join(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]



In [19]:
n = 3
df[f'n_grams_{n}'] = df['tokenized_abc'].apply(lambda tokens: generate_ngrams(tokens, n=n))


In [20]:
df.head()

,Unnamed: 0.1,Unnamed: 0,tune_id,setting_id,name,type,meter,mode,abc,date,username,full_abc,transposed_abc,tokenized_abc,n_grams_3
0,0,0,18105,35234,"$150 Boot, The",polka,2/4,Gmajor,|:d>g fe|dB AG|E/F/G E/F/G|BA GF|\nd>g fe|dB A...,2019-07-06 04:39:09,NfldWhistler,"X:35234\nT:$150 Boot, The\nM:2/4\nK:Gmajor\n|:...","X:35234\nT:$150 Boot, The\nM:2/4\nK:Cmajor\n|:...","[G_0.500, c_0.500, B_0.500, A_0.500, G_0.500, ...","[G_0.500 c_0.500 B_0.500, c_0.500 B_0.500 A_0...."
1,1,1,11931,11931,'G Iomain Nan Gamhna,slip jig,9/8,Gmajor,dBB B2 A BAG|dBB Bcd efg|dBB B2 A BAG|eAA dBG ...,2012-05-17 07:49:26,iliketurtles,X:11931\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Gmaj...,X:11931\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Cmaj...,"[G_0.500, E_0.500, E_0.500, E_1.000, D_0.500, ...","[G_0.500 E_0.500 E_0.500, E_0.500 E_0.500 E_1...."
2,2,2,11931,48772,'G Iomain Nan Gamhna,slip jig,9/8,Amixolydian,|:dBB BBA BAG|dBB Bcd efg|dBB BBA BAG|e2A dBG ...,2023-11-25 22:54:00,birlibirdie,X:48772\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Amix...,X:48772\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Cmix...,"[F_0.500, D_0.500, D_0.500, D_0.500, D_0.500, ...","[F_0.500 D_0.500 D_0.500, D_0.500 D_0.500 D_0...."
3,3,3,15326,28560,'S Ann An Ìle,strathspey,4/4,Gmajor,|:G>A B>G c>A B>G|E<E A>G F<D D2|G>A B>G c>A B...,2016-03-31 15:34:45,danninagh,X:28560\nT:'S Ann An Ìle\nM:4/4\nK:Gmajor\n|:G...,X:28560\nT:'S Ann An Ìle\nM:4/4\nK:Cmajor\n|:C...,"[C_0.500, D_0.500, E_0.500, C_0.500, F_0.500, ...","[C_0.500 D_0.500 E_0.500, D_0.500 E_0.500 C_0...."
4,4,4,15326,28582,'S Ann An Ìle,strathspey,4/4,Gmajor,"uD2|:{F}v[G,2G2]uB>ud c>A B>G|{D}E2 uA>uG F<D ...",2016-04-03 09:15:08,DonaldK,X:28582\nT:'S Ann An Ìle\nM:4/4\nK:Gmajor\nuD2...,"X:28582\nT:'S Ann An Ìle\nM:4/4\nK:Cmajor\nuG,...","[G_0.500, B_0.500, C_0.500, C_1.000, E_0.500, ...","[G_0.500 B_0.500 C_0.500, B_0.500 C_0.500 C_1...."
